In [1]:
import sys
import torch
import pandas as pd
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from transformers import AutoTokenizer, AutoModel

sys.path.append("../src/")
from preprocess import get_descriptors_2d, get_descriptors_3d, get_ecfp, MolPreprocessor
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MTR")


def get_chemberta_emb_single(
    tokenizer, model, smiles: str, device="cuda:0", max_len=256
) -> tuple[torch.Tensor]:

    input = tokenizer(
        smiles,
        max_length=max_len,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**input)
        last_hidden = outputs.last_hidden_state
        cls_emb = last_hidden[:, 0, :]
        pooled_atom_emb = last_hidden[:, 1:-1, :].mean(axis=1)
    
    return cls_emb, pooled_atom_emb

/home/tech/Hawon/dacon-jump-ai-2025/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### SMILES 로부터 descriptor, ChemBerta embedding, graph 특징 추출 및 데이터화

In [2]:
df = pd.read_pickle("../data/250714_preprocessed_HW_V2.pkl")
df.columns

Index(['smiles', 'ic50_nm', 'ic50_nm_imputed', 'is_active(10um)',
       'data_source', 'data_id', 'pvalue', 'pvalue_imputed', 'rdmol',
       'rdmol_confs5', 'medai_id'],
      dtype='object')

In [ ]:
save2dir = "../data/250716_input_preparation_hw"
mp = MolPreprocessor()

# 컬럼 인덱스 계산
smiles_idx = df.columns.get_loc('smiles')
pvalue_idx = df.columns.get_loc('pvalue')
pvalue_imputed_idx = df.columns.get_loc('pvalue_imputed')
rdmol_idx = df.columns.get_loc('rdmol_confs5')
mol_id_idx = df.columns.get_loc('medai_id')
is_active_idx = df.columns.get_loc('is_active(10um)')

for i in tqdm(range(df.shape[0])):
    smiles = df.iloc[i, smiles_idx]
    pvalue = df.iloc[i, pvalue_idx]
    pvalue_imputed = df.iloc[i, pvalue_imputed_idx]
    rd_mol = df.iloc[i, rdmol_idx]
    mol_id = df.iloc[i, mol_id_idx]
    is_active = df.iloc[i, is_active_idx]
    
    # Feature 계산
    desc_2d = get_descriptors_2d(rd_mol).unsqueeze(0)
    desc_3d = get_descriptors_3d(rd_mol).unsqueeze(0)
    ecfp = get_ecfp(rd_mol, radius=2, fpSize=2048).unsqueeze(0)
    
    # pyg Data 객체 형태로 만들기
    data = Data()

    # ChemBerta emb 추가
    cls_emb, pooled_atom_emb = get_chemberta_emb_single(tokenizer=tokenizer, model=model, smiles=smiles)
    data.chemberta_cls = cls_emb
    data.chemberta_mol = pooled_atom_emb

    # Graph feature 추가
    data.x = mp.get_lig_feature(rd_mol, to_tensor=True)
    data.edge_index, data.edge_attr = mp.get_edge_info(rd_mol)
    data.pos = mp.get_atom_position(rd_mol)
    data.z = mp.get_atomic_number(rd_mol)

    # Descriptor, FP 추가
    data.desc_2d = desc_2d
    data.desc_3d = desc_3d
    data.ecfp = ecfp

    # 저장
    torch.save(data, f"{save2dir}/{mol_id}.pt")

100%|██████████| 25753/25753 [08:46<00:00, 48.95it/s]


In [3]:
import torch

torch.load("/home/tech/Hawon/dacon-jump-ai-2025/data/preprocessed/Medai_00000.pt", weights_only=False)

/home/tech/Hawon/dacon-jump-ai-2025/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data(chemberta_cls=[1, 384], chemberta_mol=[1, 384], x=[18, 54], edge_index=[2, 38], edge_attr=[38], pos=[18, 5, 3], z=[18], desc_2d=[1, 6], desc_3d=[1, 11], ecfp=[1, 2048])